This tutorial compares two fundamental Devito DSE modes:

* `dse` = skewing
* `dse` = advanced

We will use a trivial `Operator` that, at each time step, increments by 1 all points in the physical domain and the code produced in each case.

In [1]:
def _unidiff_output(expected, actual):
    """
    Helper function. Returns a string containing the unified diff of two multiline strings.
    """

    import difflib
    expected=expected.splitlines(1)
    actual=actual.splitlines(1)

    diff=difflib.unified_diff(expected, actual)

    return ''.join(diff)

In [2]:
from devito import Grid, TimeFunction, Eq, Operator

grid = Grid(shape=(8, 8))
u_skew = TimeFunction(name='u_skew', grid=grid, save=3)
u_adv = TimeFunction(name='u_adv', grid=grid, save=3)


op_skew = Operator(Eq(u_skew.forward, u_skew + 1), dse='skewing')
#print(op_skew)
string_1 = str(op_skew)

op_adv = Operator(Eq(u_adv.forward, u_adv + 1), dse='advanced')
#print(op_adv)
string_2 = str(op_adv)

print(_unidiff_output(string_1, string_2))

Skewing pass
Factorize
--- 
+++ 
@@ -22,9 +22,9 @@
 } ;
 
 
-int Kernel(struct dataobj *restrict u_skew_vec, const int time_M, const int time_m, struct profiler * timers, const int x_M, const int x_m, const int y_M, const int y_m)
+int Kernel(struct dataobj *restrict u_adv_vec, const int time_M, const int time_m, struct profiler * timers, const int x_M, const int x_m, const int y_M, const int y_m)
 {
-  float (*restrict u_skew)[u_skew_vec->size[1]][u_skew_vec->size[2]] __attribute__ ((aligned (64))) = (float (*)[u_skew_vec->size[1]][u_skew_vec->size[2]]) u_skew_vec->data;
+  float (*restrict u_adv)[u_adv_vec->size[1]][u_adv_vec->size[2]] __attribute__ ((aligned (64))) = (float (*)[u_adv_vec->size[1]][u_adv_vec->size[2]]) u_adv_vec->data;
   /* Flush denormal numbers to zero in hardware */
   _MM_SET_DENORMALS_ZERO_MODE(_MM_DENORMALS_ZERO_ON);
   _MM_SET_FLUSH_ZERO_MODE(_MM_FLUSH_ZERO_ON);
@@ -35,10 +35,10 @@
     /* Begin section0 */
     for (int x = x_m; x <= x_M; x += 1)
     {
-   

To run `op`, we have to "`apply`" it.

In [3]:
summary = op_skew.apply()
summary = op_adv.apply()

Operator `Kernel` run in 0.00 s
Operator `Kernel` run in 0.00 s


Under-the-hood, some code has been generated (`print(op)` to display the generated code), JIT-compiled, and executed. Since no additional arguments have been passed, `op` has used `u` as input. We can verify that the content of `u.data` is as expected

In [4]:
print(u_skew.dimensions, u_skew.shape)
print("----------")
print(u_adv.dimensions, u_adv.shape)

(time, x, y) (3, 8, 8)
----------
(time, x, y) (3, 8, 8)


In [5]:
print(u_skew.data)
print("----------")
print(u_adv.data)

[[[0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]]

 [[1. 1. 1. 1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1. 1. 1. 1.]]

 [[2. 2. 2. 2. 2. 2. 2. 2.]
  [2. 2. 2. 2. 2. 2. 2. 2.]
  [2. 2. 2. 2. 2. 2. 2. 2.]
  [2. 2. 2. 2. 2. 2. 2. 2.]
  [2. 2. 2. 2. 2. 2. 2. 2.]
  [2. 2. 2. 2. 2. 2. 2. 2.]
  [2. 2. 2. 2. 2. 2. 2. 2.]
  [2. 2. 2. 2. 2. 2. 2. 2.]]]
----------
[[[0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]]

 [[1. 1. 1. 1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1. 1. 1. 1.]
 

In particular, we observe that:

* `u` has size 3 along the time dimension, since it was built with `save=3`. Therefore `op` could only execute 2 timesteps, namely time=0 and time=1; given `Eq(u.forward, u + 1)`, executing time=2 would cause out-of-bounds access errors. Devito figures this out automatically and sets appropriate minimum and maximum iteration points.
* All 16 points in each timeslice of the 4x4 `Grid` have been computed.

To access all default arguments used by `op` *without* running the `Operator`, one can run

In [6]:
print(op_skew.arguments())
print(op_adv.arguments())

{'u_skew': <cparam 'P' (0x7f4511fb9768)>, 'time_m': 0, 'time_size': 3, 'time_M': 1, 'x_m': 0, 'x_size': 8, 'x_M': 7, 'y_m': 0, 'y_size': 8, 'y_M': 7, 'timers': <cparam 'P' (0x7f451258de68)>}
{'u_adv': <cparam 'P' (0x7f4511fb97d8)>, 'time_m': 0, 'time_size': 3, 'time_M': 1, 'x_m': 0, 'x_size': 8, 'x_M': 7, 'y_m': 0, 'y_size': 8, 'y_M': 7, 'timers': <cparam 'P' (0x7f45124e2ef0)>}


`'u'` stores a pointer to the allocated data; `'timers'` stores a pointer to a data structure used for C-level performance profiling.

One may want to replace some of these default arguments. For example, we could increase the minimum iteration point along the spatial Dimensions `x` and `y`, and execute only the very first timestep:

In [7]:
u_skew.data[:] = 0.  # Explicit reset to initial value
summary = op_skew.apply(x_m=2, y_m=2, time_M=0)

Operator `Kernel` run in 0.00 s


We look again at the computed data to convince ourselves that everything went as intended to go

In [8]:
u_skew.data

Data([[[0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.]],

      [[0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 1., 1., 1., 1., 1.],
       [0., 0., 1., 1., 1., 1., 1., 1.],
       [0., 0., 1., 1., 1., 1., 1., 1.],
       [0., 0., 1., 1., 1., 1., 1., 1.],
       [0., 0., 1., 1., 1., 1., 1., 1.],
       [0., 0., 1., 1., 1., 1., 1., 1.]],

      [[0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.]]], dtype=floa

Given a generic `Dimension` `d`, the naming convention is such that:

* `d_m` is the minimum iteration point
* `d_M` is the maximum iteration point

Hence, `op.apply(..., d_m=4, d_M=7, ...)` will run `op` in the compact interval `[4, 7]` along `d`. For historical reasons, `d=...` aliases to `d_M=...`; in many Devito examples it happens to see `op.apply(..., time=10, ...)` -- this is just equivalent to `op.apply(..., time_M=10, ...)`.

If we try to specify an invalid iteration extreme, Devito will raise an exception.

In [10]:
from devito.exceptions import InvalidArgument
try:
    op_skew.apply(time_M=2)
except InvalidArgument as e:
    print(e)

OOB detected due to time_M=2


The same `Operator` can be applied to a different `TimeFunction`. For example:

In [13]:
u2 = TimeFunction(name='u', grid=grid, save=5)
summary = op_skew.apply(u_skew = u2)
u2.data

Operator `Kernel` run in 0.00 s


Data([[[0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.]],

      [[1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1.]],

      [[2., 2., 2., 2., 2., 2., 2., 2.],
       [2., 2., 2., 2., 2., 2., 2., 2.],
       [2., 2., 2., 2., 2., 2., 2., 2.],
       [2., 2., 2., 2., 2., 2., 2., 2.],
       [2., 2., 2., 2., 2., 2., 2., 2.],
       [2., 2., 2., 2., 2., 2., 2., 2.],
       [2., 2., 2., 2., 2., 2., 2., 2.],
       [2., 2., 2., 2., 2., 2., 2., 2.]],

      [[3.

Note that this was the third call to `op.apply`, but code generation and JIT-compilation only occurred upon the very first call.

There is one relevant case in which the maximum iteration point along the time dimension must be specified -- whenever `save` is unset, as in such a case the `Operator` wouldn't know for how many iterations to run.

In [14]:
v = TimeFunction(name='v', grid=grid)
op2 = Operator(Eq(v.forward, v + 1))
try:
    op2.apply()
except ValueError as e:
    print(e)

No value found for parameter time_M


In [15]:
summary = op2.apply(time_M=4)
v.data

Operator `Kernel` run in 0.00 s


Data([[[4., 4., 4., 4., 4., 4., 4., 4.],
       [4., 4., 4., 4., 4., 4., 4., 4.],
       [4., 4., 4., 4., 4., 4., 4., 4.],
       [4., 4., 4., 4., 4., 4., 4., 4.],
       [4., 4., 4., 4., 4., 4., 4., 4.],
       [4., 4., 4., 4., 4., 4., 4., 4.],
       [4., 4., 4., 4., 4., 4., 4., 4.],
       [4., 4., 4., 4., 4., 4., 4., 4.]],

      [[5., 5., 5., 5., 5., 5., 5., 5.],
       [5., 5., 5., 5., 5., 5., 5., 5.],
       [5., 5., 5., 5., 5., 5., 5., 5.],
       [5., 5., 5., 5., 5., 5., 5., 5.],
       [5., 5., 5., 5., 5., 5., 5., 5.],
       [5., 5., 5., 5., 5., 5., 5., 5.],
       [5., 5., 5., 5., 5., 5., 5., 5.],
       [5., 5., 5., 5., 5., 5., 5., 5.]]], dtype=float32)

The `summary` variable can be inspected to retrieve performance metrics.

In [16]:
summary

PerformanceSummary([('section0',
                     PerfEntry(time=1e-06, gflopss=0.0, gpointss=0.0, oi=0.0, ops=0, itershapes=[]))])

We observe that basically all entries except for the execution time are fixed at 0. This is because by default Devito avoids to compute performance metrics, to minimize the processing time before returning control to the user (in complex `Operators`, the processing time to retrieve, for instance, the operation count or the memory footprint could be significant). To compute all performance metrics, a user could either export the environment variable `DEVITO_PROFILING` to `'advanced'` or change the profiling level programmatically *before* the `Operator` is constructed

In [18]:
from devito import configuration
configuration['profiling'] = 'advanced'

op = Operator(Eq(u_skew.forward, u_skew*u_skew + 1))
op.apply()

Operator `Kernel` run in 0.00 s


PerformanceSummary([('section0',
                     PerfEntry(time=2e-06, gflopss=0.064, gpointss=0.064, oi=0.08333333333333333, ops=1, itershapes=[(2, 8, 8)]))])